In [2]:
# Imports,
import os
import torch
import torchaudio
import random
import librosa
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import Resample
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# Configuration
processed_dir = "VocalSet_processed"
splits = ['train', 'val', 'test']
sample_rate = 22050
duration = 4.0  # seconds
data = {}  # To store file paths or audio data

# Access and optionally load files
for split in splits:
    split_path = os.path.join(processed_dir, split)
    file_paths = [os.path.join(split_path, f) for f in os.listdir(split_path) if f.endswith('.wav')]
    
    # Optionally: Load audio files
    audio_data = []
    for path in file_paths:
        try:
            y, _ = librosa.load(path, sr=sample_rate)
            audio_data.append((path, y))  # or just y if you don’t need the path
        except Exception as e:
            print(f"Could not load {path}: {e}")
    
    data[split] = audio_data  # Contains (path, audio_array) for each file

# Example usage
print(f"Train files loaded: {len(data['train'])}")
print(f"Validation files loaded: {len(data['val'])}")
print(f"Test files loaded: {len(data['test'])}")


Train files loaded: 2890
Validation files loaded: 361
Test files loaded: 362


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import soundfile as sf

# CONFIG
DATA_DIR = 'VocalSet_processed'
SAMPLE_RATE = 22050
DURATION = 4.0
AUDIO_LENGTH = int(SAMPLE_RATE * DURATION)
BATCH_SIZE = 16
EPOCHS = 100
LATENT_DIM = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_DIR = "gan_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==== Dataset ====
class AudioDataset(Dataset):
    def __init__(self, split_dir):
        self.files = [os.path.join(split_dir, f) for f in os.listdir(split_dir) if f.endswith(".wav")]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        audio, _ = librosa.load(path, sr=SAMPLE_RATE)
        audio = librosa.util.fix_length(audio, size=AUDIO_LENGTH)
        return torch.tensor(audio, dtype=torch.float32)

# ==== Models ====
class Generator(nn.Module):
    def __init__(self, latent_dim, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, output_size),
            nn.Tanh()
        )


    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        return self.model(x)

# ==== DataLoader ====
train_dataset = AudioDataset(os.path.join(DATA_DIR, "train"))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# ==== Initialize Models ====
generator = Generator(LATENT_DIM, AUDIO_LENGTH).to(DEVICE)
discriminator = Discriminator(AUDIO_LENGTH).to(DEVICE)

# ==== Optimizers and Loss ====
criterion = nn.BCEWithLogitsLoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# ==== Training Loop ====
for epoch in range(EPOCHS):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        batch = batch.to(DEVICE)

        real_labels = torch.full((BATCH_SIZE, 1), 0.9).to(DEVICE)  # label smoothing
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)

        # === Train Discriminator ===
        optimizer_D.zero_grad()
        outputs_real = discriminator(batch)
        loss_real = criterion(outputs_real, real_labels)

        z = torch.randn(BATCH_SIZE, LATENT_DIM).to(DEVICE)
        fake_audio = generator(z)
        outputs_fake = discriminator(fake_audio.detach())
        loss_fake = criterion(outputs_fake, fake_labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), 1.0)
        optimizer_D.step()

        # === Train Generator ===
        optimizer_G.zero_grad()
        outputs = discriminator(fake_audio)
        loss_G = criterion(outputs, real_labels)
        loss_G.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

    # === Save sample audio ===
    if (epoch + 1) % 10 == 0:
        generator.eval()
        with torch.no_grad():
            z = torch.randn(1, LATENT_DIM).to(DEVICE)
            gen_audio = generator(z).cpu().numpy().flatten()
            sf.write(f"{OUTPUT_DIR}/sample_epoch_{epoch+1}.wav", gen_audio, SAMPLE_RATE)
        generator.train()

# ==== Save models ====
torch.save(generator.state_dict(), os.path.join(OUTPUT_DIR, "generator.pth"))
torch.save(discriminator.state_dict(), os.path.join(OUTPUT_DIR, "discriminator.pth"))
print("Models saved.")


Epoch 1/100: 100%|██████████| 180/180 [17:55<00:00,  5.97s/it]


Epoch [1/100] Loss_D: 2.7504, Loss_G: 82.0034


Epoch 2/100: 100%|██████████| 180/180 [17:07<00:00,  5.71s/it]


Epoch [2/100] Loss_D: 0.4604, Loss_G: 84.0246


Epoch 3/100: 100%|██████████| 180/180 [17:20<00:00,  5.78s/it]


...

Epoch [43/100] Loss_D: 0.3340, Loss_G: 20.0828


Epoch 44/100: 100%|██████████| 180/180 [18:15<00:00,  6.08s/it]


Epoch [44/100] Loss_D: 0.3260, Loss_G: 33.2990


Epoch 45/100:  26%|██▌       | 47/180 [05:05<14:24,  6.50s/it]


KeyboardInterrupt: 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import soundfile as sf
from torch.cuda.amp import autocast, GradScaler

# === CONFIG ===
DATA_DIR = 'VocalSet_processed'
SAMPLE_RATE = 16000  # Reduced sample rate
DURATION = 2.0  # Reduced duration to 2 seconds
AUDIO_LENGTH = int(SAMPLE_RATE * DURATION)
BATCH_SIZE = 32
EPOCHS = 50
LATENT_DIM = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_DIR = "gan_outputs_improved"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Dataset ===
class AudioDataset(Dataset):
    def __init__(self, split_dir):
        self.files = [os.path.join(split_dir, f) for f in os.listdir(split_dir) if f.endswith(".wav")]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        waveform, sr = torchaudio.load(path)
        waveform = torch.mean(waveform, dim=0)  # mono
        if sr != SAMPLE_RATE:
            waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=SAMPLE_RATE)
        waveform = torch.nn.functional.pad(waveform, (0, max(0, AUDIO_LENGTH - waveform.size(0))))
        waveform = waveform[:AUDIO_LENGTH]
        return waveform

# === Generator ===
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256 * 64),
            nn.ReLU(True),
            nn.Unflatten(1, (256, 64)),
            nn.ConvTranspose1d(256, 128, 25, stride=4, padding=11, output_padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.ConvTranspose1d(128, 64, 25, stride=4, padding=11, output_padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            nn.ConvTranspose1d(64, 1, 25, stride=2, padding=12, output_padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.model(z)
        x = x.squeeze(1)
        if x.size(1) < AUDIO_LENGTH:
            x = torch.nn.functional.pad(x, (0, AUDIO_LENGTH - x.size(1)))
        return x[:, :AUDIO_LENGTH]

# === Discriminator ===
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(1, 32, 25, stride=4, padding=11),  # Reduced number of channels
            nn.LeakyReLU(0.2),
            nn.Conv1d(32, 64, 25, stride=4, padding=11),
            nn.LeakyReLU(0.2),
            nn.Conv1d(64, 128, 25, stride=4, padding=11),
            nn.LeakyReLU(0.2),
        )
        
        # Compute output size dynamically
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, AUDIO_LENGTH)
            out = self.features(dummy_input)
            self.flattened_size = out.view(1, -1).size(1)

        self.classifier = nn.Linear(self.flattened_size, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, AUDIO_LENGTH]
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# === Gradient Penalty ===
def gradient_penalty(D, real, fake):
    batch_size = real.size(0)
    alpha = torch.rand(batch_size, 1).to(DEVICE)
    alpha = alpha.expand_as(real)

    interpolates = alpha * real + (1 - alpha) * fake
    interpolates = interpolates.requires_grad_(True)

    d_interpolates = D(interpolates)
    ones = torch.ones_like(d_interpolates).to(DEVICE)

    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=ones,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    gradients = gradients.view(batch_size, -1)
    penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return penalty

# === DataLoader ===
train_dataset = AudioDataset(os.path.join(DATA_DIR, "train"))
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    drop_last=True, 
    num_workers=8,  # Increased number of workers
    pin_memory=True  # Enable memory pinning for faster transfer to GPU
)

# === Model Initialization ===
generator = Generator(LATENT_DIM).to(DEVICE)
discriminator = Discriminator().to(DEVICE)

# === Optimizers ===
optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))

# === Mixed Precision Training ===
scaler = GradScaler()

# === Training Loop ===
lambda_gp = 10
n_critic = 1  # Reduced from 2 to 1 to speed up training

for epoch in range(EPOCHS):
    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")):
        real_audio = batch.to(DEVICE)

        # Train Discriminator
        z = torch.randn(BATCH_SIZE, LATENT_DIM).to(DEVICE)
        fake_audio = generator(z).detach()

        for _ in range(n_critic):
            with autocast():
                d_real = discriminator(real_audio)
                d_fake = discriminator(fake_audio)
                gp = gradient_penalty(discriminator, real_audio, fake_audio)
                loss_D = -(torch.mean(d_real) - torch.mean(d_fake)) + lambda_gp * gp

            scaler.scale(loss_D).backward()
            scaler.step(optimizer_D)
            scaler.update()

        # Train Generator
        z = torch.randn(BATCH_SIZE, LATENT_DIM).to(DEVICE)
        fake_audio = generator(z)

        with autocast():
            loss_G = -torch.mean(discriminator(fake_audio))

        scaler.scale(loss_G).backward()
        scaler.step(optimizer_G)
        scaler.update()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

    if (epoch + 1) % 10 == 0:
        generator.eval()
        with torch.no_grad():
            z = torch.randn(1, LATENT_DIM).to(DEVICE)
            gen_audio = generator(z).cpu().numpy().flatten()
            sf.write(f"{OUTPUT_DIR}/sample_epoch_{epoch+1}.wav", gen_audio, SAMPLE_RATE)
        generator.train()

# === Save Models ===
torch.save(generator.state_dict(), os.path.join(OUTPUT_DIR, "generator.pth"))
torch.save(discriminator.state_dict(), os.path.join(OUTPUT_DIR, "discriminator.pth"))
print("Models saved.")


C:\Users\DELL\AppData\Local\Temp\ipykernel_27324\4250236396.py:139: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
c:\Users\DELL\OneDrive\Documents\FAST\SEMESTER VIII\GenerativeAI\Project\myenv\Lib\site-packages\torch\amp\grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/50:   0%|          | 0/90 [00:00<?, ?it/s]c:\Users\DELL\OneDrive\Documents\FAST\SEMESTER VIII\GenerativeAI\Project\myenv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import soundfile as sf
import random

# CONFIG
DATA_DIR = 'VocalSet_processed'
SAMPLE_RATE = 22050
DURATION = 4.0
AUDIO_LENGTH = int(SAMPLE_RATE * DURATION)
BATCH_SIZE = 16
EPOCHS = 100
LATENT_DIM = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_DIR = "gan_outputs_improved"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==== Dataset ====
class AudioDataset(Dataset):
    def __init__(self, split_dir):
        self.files = [os.path.join(split_dir, f) for f in os.listdir(split_dir) if f.endswith(".wav")]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        audio, _ = librosa.load(path, sr=SAMPLE_RATE)
        audio = librosa.util.fix_length(audio, size=AUDIO_LENGTH)
        return torch.tensor(audio, dtype=torch.float32)

# ==== Models ====
class Generator(nn.Module):
    def __init__(self, latent_dim, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, output_size),
            nn.Tanh()
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.utils.spectral_norm(nn.Linear(input_size, 2048)),
            nn.LeakyReLU(0.2),
            nn.utils.spectral_norm(nn.Linear(2048, 1024)),
            nn.LeakyReLU(0.2),
            nn.utils.spectral_norm(nn.Linear(1024, 1))
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x):
        return self.model(x)

# ==== DataLoader ====
train_dataset = AudioDataset(os.path.join(DATA_DIR, "train"))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# ==== Initialize Models ====
generator = Generator(LATENT_DIM, AUDIO_LENGTH).to(DEVICE)
discriminator = Discriminator(AUDIO_LENGTH).to(DEVICE)

# ==== Optimizers and Loss ====
criterion = nn.BCEWithLogitsLoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Learning Rate Schedulers
scheduler_G = optim.lr_scheduler.StepLR(optimizer_G, step_size=30, gamma=0.5)
scheduler_D = optim.lr_scheduler.StepLR(optimizer_D, step_size=30, gamma=0.5)

# ==== Training Loop ====
for epoch in range(EPOCHS):
    total_loss_D = 0.0
    total_loss_G = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        batch = batch.to(DEVICE)

        # Label smoothing and flipping
        real_labels = torch.full((BATCH_SIZE, 1), 0.9).to(DEVICE)
        fake_labels = torch.zeros((BATCH_SIZE, 1)).to(DEVICE)

        if random.random() < 0.05:  # Randomly flip labels (5% chance)
            real_labels, fake_labels = fake_labels, real_labels

        # === Train Discriminator ===
        optimizer_D.zero_grad()
        outputs_real = discriminator(batch)
        loss_real = criterion(outputs_real, real_labels)

        z = torch.randn(BATCH_SIZE, LATENT_DIM).to(DEVICE)
        fake_audio = generator(z)
        outputs_fake = discriminator(fake_audio.detach())
        loss_fake = criterion(outputs_fake, fake_labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), 1.0)
        optimizer_D.step()

        # === Train Generator ===
        optimizer_G.zero_grad()
        outputs = discriminator(fake_audio)
        loss_G = criterion(outputs, real_labels)
        loss_G.backward()
        torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
        optimizer_G.step()

        total_loss_D += loss_D.item()
        total_loss_G += loss_G.item()

    scheduler_G.step()
    scheduler_D.step()

    avg_loss_D = total_loss_D / len(train_loader)
    avg_loss_G = total_loss_G / len(train_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Avg Loss_D: {avg_loss_D:.4f}, Avg Loss_G: {avg_loss_G:.4f}")

    # === Save sample audio ===
    if (epoch + 1) % 10 == 0:
        generator.eval()
        with torch.no_grad():
            z = torch.randn(1, LATENT_DIM).to(DEVICE)
            gen_audio = generator(z).cpu().numpy().flatten()
            sf.write(f"{OUTPUT_DIR}/sample_epoch_{epoch+1}.wav", gen_audio, SAMPLE_RATE)
        generator.train()

# ==== Save models ====
torch.save(generator.state_dict(), os.path.join(OUTPUT_DIR, "generator.pth"))
torch.save(discriminator.state_dict(), os.path.join(OUTPUT_DIR, "discriminator.pth"))
print("Models saved.")


Epoch 1/100: 100%|██████████| 180/180 [49:19<00:00, 16.44s/it]


Epoch [1/100] Avg Loss_D: 0.9690, Avg Loss_G: 6.4710


Epoch 2/100: 100%|██████████| 180/180 [38:38<00:00, 12.88s/it]


Epoch [2/100] Avg Loss_D: 0.8485, Avg Loss_G: 6.8569


Epoch 3/100: 100%|██████████| 180/180 [34:34<00:00, 11.52s/it]


...

Epoch 43/100: 100%|██████████| 180/180 [35:47<00:00, 11.93s/it]


Epoch [43/100] Avg Loss_D: 1.1377, Avg Loss_G: 0.9835


Epoch 44/100: 100%|██████████| 180/180 [1:03:02<00:00, 21.02s/it]


Epoch [44/100] Avg Loss_D: 1.1266, Avg Loss_G: 0.9822


Epoch 45/100:  68%|██████▊   | 122/180 [37:51<11:52, 12.29s/it] 